### 1 - Imports

In [1]:

import unittest
import pandas as pd
import numpy as np

import sys
from pathlib import Path

ROOT_DIR = Path.cwd().parent
sys.path.append(str(ROOT_DIR))

from src.feature_engineering import extract_features

 


### 2 - Exemple de dataframe de test :

In [2]:

data = pd.DataFrame({
    "VS1": [1, 2, 3],
    "PS1": [4, 5, 6],
    "PS2": [7, 8, 9],
    "SE": [10, 20, 30],
    "Unknown": [100, 200, 300]  
})


Dans ce script On crée des données contrôlées pour tester toutes les fonctionnalités.

On inclut aussi une colonne hors sensor_groups pour tester qu’elle n’est pas traitée.

### 04 - Test 1 : Les colonnes calculées existent

In [3]:
class TestExtractFeatures(unittest.TestCase):

    def test_feature_columns_exist(self):
        features_df = extract_features(data)
        expected_cols = [
            "VS1_mean", "VS1_std", "VS1_max", "VS1_median",
            "PS1_mean", "PS1_std", "PS1_max", "PS1_median",
            "PS2_mean", "PS2_std", "PS2_max", "PS2_median",
            "SE_mean", "SE_std", "SE_max", "SE_median"
        ]
        for col in expected_cols:
            self.assertIn(col, features_df.columns)
        # Colonne inconnue ne doit pas apparaître
        self.assertNotIn("Unknown_mean", features_df.columns)


In [4]:

suite = unittest.TestLoader().loadTestsFromTestCase(TestExtractFeatures)
unittest.TextTestRunner(verbosity=2).run(suite)


test_feature_columns_exist (__main__.TestExtractFeatures.test_feature_columns_exist) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Ce script permet de Vérifie :

- Les colonnes attendues sont bien créées

- Les colonnes non définies dans sensor_groups sont ignorées

### 05 - Test 2 : Valeurs correctes

In [5]:

class TestExtractFeatures(unittest.TestCase):

    def test_feature_values(self):
        features_df = extract_features(data)

        # VS1
        self.assertEqual(features_df["VS1_mean"].iloc[0], 2)
        self.assertAlmostEqual(features_df["VS1_std"].iloc[0], np.std([1,2,3], ddof=1))
        self.assertEqual(features_df["VS1_max"].iloc[0], 3)
        self.assertEqual(features_df["VS1_median"].iloc[0], 2)

        # SE
        self.assertEqual(features_df["SE_mean"].iloc[0], 20)
        self.assertEqual(features_df["SE_max"].iloc[0], 30)
        self.assertEqual(features_df["SE_median"].iloc[0], 20)


In [6]:

suite = unittest.TestLoader().loadTestsFromTestCase(TestExtractFeatures)
unittest.TextTestRunner(verbosity=2).run(suite)


test_feature_values (__main__.TestExtractFeatures.test_feature_values) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Cette fonction permet de vérifie que les calculs de mean, std, max, median sont corrects.

### 06 - Test 3 : Robustesse sur DataFrame vide ou sans colonnes valides

In [7]:

class TestExtractFeatures(unittest.TestCase):

    def test_empty_dataframe(self):
        empty_df = pd.DataFrame()
        features_df = extract_features(empty_df)
        # On doit avoir un DataFrame avec 1 ligne mais aucune colonne
        self.assertEqual(features_df.shape[0], 1)
        self.assertEqual(features_df.shape[1], 0)

    def test_no_valid_columns(self):
        df = pd.DataFrame({"X": [1,2,3]})
        features_df = extract_features(df)
        self.assertEqual(features_df.shape[1], 0)


In [8]:

suite = unittest.TestLoader().loadTestsFromTestCase(TestExtractFeatures)
unittest.TextTestRunner(verbosity=2).run(suite)


test_empty_dataframe (__main__.TestExtractFeatures.test_empty_dataframe) ... ok
test_no_valid_columns (__main__.TestExtractFeatures.test_no_valid_columns) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.012s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

Ici, on vérifie que la fonction ne plante jamais même si les colonnes ne correspondent pas.

### 07 - Test 4 : Pas de NaN générés

In [ ]:

class TestExtractFeatures(unittest.TestCase):

    def test_no_nan_in_output(self):
        features_df = extract_features(data)
        self.assertFalse(features_df.isna().any().any())


In [9]:

suite = unittest.TestLoader().loadTestsFromTestCase(TestExtractFeatures)
unittest.TextTestRunner(verbosity=2).run(suite)


test_empty_dataframe (__main__.TestExtractFeatures.test_empty_dataframe) ... ok
test_no_valid_columns (__main__.TestExtractFeatures.test_no_valid_columns) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

ce test garantit que la fonction ne produit pas de NaN si les données d’entrée sont correctes.